In [1]:
!pip install --upgrade nbformat ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


# **scispacy:**

In [ ]:
import spacy

# Load NER-specific model
nlp = spacy.load("en_ner_bc5cdr_md")

text = "Patient was prescribed aspirin for hypertension"
doc = nlp(text)

print("Medical Entities found:")
for ent in doc.ents:
    print(ent.text, ent.label_)


Medical Entities found:
aspirin CHEMICAL
hypertension DISEASE


# **ClinicalBERT:**

In [ ]:
# Install transformers and torch if not already installed
# !pip install transformers torch

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load ClinicalBERT model (fine-tuned on clinical notes)
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Medical text
text = "Patient shows symptoms of type 2 diabetes and hypertension"

# Run NER
entities = ner_pipeline(text)

# Print entities
print("ClinicalBERT Medical Entities found:")
for ent in entities:
    print(ent['word'], ent['entity_group'])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ClinicalBERT Medical Entities found:
patient shows LABEL_0
symptoms LABEL_1
of type LABEL_0
2 LABEL_1
diabetes LABEL_0
and LABEL_1
hypertens LABEL_0
##ion LABEL_1


# **Regex Patterns for Extracting Numbers + Units**

# **1️.Basic Number Extraction**

In [ ]:
import re

text = "Patient was given 500 mg paracetamol twice daily for 5 days"
numbers = re.findall(r'\d+\.?\d*', text)
print(numbers)


['500', '5']


## **2️.Number + Unit Extraction**

In [ ]:
pattern = r'\b\d+\.?\d*\s?(mg|ml|g|kg|units|mcg|%)\b'
matches = re.findall(pattern, text, re.IGNORECASE)
print(matches)


['mg']


# **3️.Full Dosage Extraction**

In [ ]:
pattern = r'\b(\d+\.?\d*)\s?(mg|ml|units|mcg)\b'
matches = re.findall(pattern, text)
print(matches)


[('500', 'mg')]


# **4️.Frequency Extraction**

In [ ]:
freq_pattern = r'\b(once|twice|thrice|\d+\s?times)\s?(daily|a day)\b'
re.findall(freq_pattern, text, re.IGNORECASE)


[('twice', 'daily')]

In [ ]:
text = "Insulin 2 units subcutaneously twice daily"

dose = re.findall(r'\b\d+\s?(units|mg|ml)\b', text)
freq = re.findall(r'\b(once|twice)\s?daily\b', text)

print(dose, freq)


['units'] ['twice']


In [ ]:
!pip install word2number

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=877de53daafbe7a6da281c34b7302fd9aa0d7e8789929a2bad64272883182eb5
  Stored in directory: /root/.cache/pip/wheels/5b/79/fb/d25928e599c7e11fe4e00d32048cd74933f34a74c633d2aea6
Successfully built word2number


# **Normalization Before Evaluation**

# **1.Convert Number Words to Digits**

In [ ]:
from word2number import w2n

text = "two milligrams"
number = w2n.word_to_num("two")
print(number)


2


# **2.Normalize Units Using Mapping**

In [ ]:
unit_map = {
    "milligrams": "mg",
    "milligram": "mg",
    "micrograms": "mcg",
    "grams": "g"
}

text = "two milligrams"
for k, v in unit_map.items():
    text = text.replace(k, v)

print(text)


two mg


# **3.Combine Number + Unit**

In [ ]:
normalized = f"{number} mg"
print(normalized)


2 mg


In [ ]:
import re

text = "The patient will return in 2 weeks"

pattern = r"\b\d+\s+(day|days|week|weeks|month|months)\b"
matches = re.findall(pattern, text)

print(matches)


['weeks']
